# Agentic Enterprise Knowledge Agent

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List, Optional
from langchain.schema import Document

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

In [ ]:
class RAGState(TypedDict):
    question: str
    query_type: str
    retrieved_docs: List[Document]
    search_query: str
    answer: str

In [ ]:
def build_faiss_index(docs_path:str)->FAISS:
    loader = DirectoryLoader(docs_path, glob="**/*")
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlab=150,
    )

    chunks = splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(chunks, embeddings)